# K-way Merge Sort

**Instructions**. Complete the following:

    Implement three-way merge sort in Python. It should at a minimum accept lists of integers as input.

    Implement a second version of three-way merge sort that calls insertion sort when sublists are below a certain length (of your choice) rather than continuing the subdivision process.

    (Optional challenge) Implement k-way merge sort, where the user specifies k. Develop and run experiments to support a hypothesis about the “best” value of k.

    Analyze and compare the practical run times of regular merge sort, three-way merge sort, and the augmented merge sort from (2). Use the tools developed in class to run experiments. Your results should be presented in a table, along with an explanatory paragraph and any useful graphs or other charts. Part of your analysis should indicate whether or not there is a “best” variation.

I implement k-way merge sort below. I analyze the running time complexity of the merge sort procedure and argue that for a small $k$, the algorithm remains $O(n*\log n)$. However, for $k = n$, the procedure becomes $O(n^2)$.

In [1]:
def merge(arr, l, mids, r):
    
    '''
    Merge is a subprocedure of mergeSort, declared below. It 
    merges k sorted subarrays into a sorted array. It does so by
    iterating continuously over all subarrays, finding the 
    smallest key, and relocating it to the right position of 
    the outcome array.
    Inputs:
    
        arr (list) The entire array to be sorted.
        
        l, r (int) The left and right indexes that define the 
          specific section of the array that should be sorted 
          in the current routine of merge().
        
        mids (list) The mid-points in the array arr[l, r] that 
          define the sorted subarrays that need to be merged. 
          The length of the list + 1 represents the "k" of the 
          merge sort. For example, if there are two mid points, 
          the procedure is a 3-way merge.     
    '''
    
    # Create a list of the size of the subarrays.
    sizes = [mids[0] - l + 1]
    for mid_index in range(1, len(mids)):
        sizes.append(mids[mid_index] - mids[mid_index - 1])
    sizes.append(r - mids[-1])
    
    # Create temporary arrays with the appropriate sizes.
    temp_arrays = [[0]*sizes[i] for i in range(len(sizes))]

    # Copy the keys into the temporary arrays.
    for i in range(0, sizes[0]):
        temp_arrays[0][i] = arr[l + i]
        
    for index_array in range(1, len(temp_arrays)):
        for index_key in range(0, sizes[index_array]):
            temp_arrays[index_array][index_key] = 
            arr[mids[index_array - 1] + index_key + 1]

    # Append a sentinel to the end of the temporary arrays.        
    for array in temp_arrays:
        array.append(float("inf"))

    # Create an array of indices to store the index of the 
    # next key of each subarray to be evaluated.
    temp_indices = [0 for i in range(len(temp_arrays))] 
    for key in range(l, r+1):
        
        # Start with a guess: the first subarray contains the 
        # next sorted key
        this_min = temp_arrays[0][temp_indices[0]]
        arr[key] = this_min
        temp_indices[0] += 1 
        index_last_array = 0

        # Check the guess, adding or subtracing from array 
        # indices as needed
        for array_index in range(1, len(temp_arrays)):
            this_key = temp_arrays[array_index][temp_indices[array_index]]
            if this_key < this_min:
                this_min = this_key
                temp_indices[array_index] += 1
                temp_indices[index_last_array] -= 1
                index_last_array = array_index
        
        # Put the current smallest element at the right 
        # part of the array
        arr[key] = this_min            

def mergeSort(arr, l, r, k = 2):   
    '''
    k-way merge sort solves the sorting problem recursively 
    by calling itself on k subproblems of smaller size by an
    order of k, approximately. Its base case is a unitary list,
    which is trivially sorted, when the merging procedure returns.
    The merge procedure is then called to merge the sorted subarrays.
    Inputs:
    
        arr (list) The entire array to be sorted.
        
        l, r (int) The left and right indexes that define the 
          specific section of the array that should be sorted
          in the current routine of mergeSort().
        
        k (int) The number of subproblems that the procedure 
          should create. As k increases, the recursion tree 
          gains breadth and becomes shorter. Default: 2.
    '''
    
    
    # Calculate the largest well-defined k-merge for the input 
    # array and indices
    largest_k = r - l + 1
    if largest_k <= 1:
        return 
    else:
        # If the user-defined k is larger than the size of the array, 
        # set k to the default.
        if largest_k < k:
            k = 2
        
        # Calculate the relevant mid points.
        mid_points = [((_*(r - l)) // k) + l for _ in range(1, k)]
        
        # Call mergeSort on the k subproblems.
        mergeSort(arr, l, mid_points[0], k)
        mergeSort(arr, mid_points[-1] + 1, r, k)       
        for mid_index in range(1, len(mid_points)):
            mergeSort(arr, mid_points[mid_index - 1] + 1, 
                      mid_points[mid_index], k)
        
        merge(arr, l, mid_points, r)

### Complexity of merge

Notice the nested for loops at the end of the merge procedure. The outer loop iterates over the keys in the input array, thus running $n$ times. The inner loop iterates over the relevant keys of the temporary subarrays. We have $k$ subarrays in a k-way merge sort; thus the loop will run $k$ times. Therefore, and given we can ignore the other operations as they will not be as expensive complexity-wise, the merge procedure is $\Theta(k*n)$. If $k$ is a constant, such as 2 in the binary merge sort, the complexity is $\Theta(n)$. However, in the special case in which we declare $k$ to be equal to $n$, our merge procedure is $\Theta(n^2)$. This special case is similar to selection sort, where we continuously find the ith smallest element in the array and place it in the ith position. The difference here is that the selection sort is an in-place sorting, whereas in the merge procedure we create auxiliary arrays to help with the sorting (thus not needing to swap keys as in selection sort). In a general case where we do not know the value of $k$, our merge algorithm is $\Omega(n)$ and $O(n^2)$.

### Complexity of mergeSort

K-way merge sort divides the initial array to be sorted into $k$ subarrays, each one $k$ times smaller than the initial input. If we define $k$ as a constant much smaller than $n$, we saw above that the complexity of the merge procedure should be $\Theta(n)$. Therefore, we arrive at the following recursive relation: $$T(n) = k*T(n/k) + \Theta(n).$$ The easiest way to solve this relation is to apply the second case of the master theorem. We have $a = b = k$, therefore: $$n^{\log_ba} = n^{\log_kk} = n.$$ Because $f(n) = \Theta(n) = \Theta(n^{\log_ba})$, we know from the master theorem that $T(n) = \Theta(n^{\log_ba}*\log n) = \Theta(n*\log n)$. Therefore, for small $k$, we should not expect the running time complexity of merge sort to vary with $k$. It is possible, however, that the constants vary predictably such as to show a certain value of $k$ to be superior to others. A hypothetical cause could be that as we increase $k$, we decrease the recursive calls made to mergeSort. We increase, however, the complexity of the merge procedure as we need to perform k comparisons before finding the correct key. We will test these hypotheses experimentally further below.

In [2]:
def mixedSort(arr, l, r, k = 2):
    '''
    The algorithm calls insertion sort for input sizes smaller than 100. 
    For larger inputs, it calls te mergeSort procedure declared above. 
    '''
    # Calculate the largest well-defined k-merge for the input array 
    # and indices. largest_k is equal to the length of the subarray.
    largest_k = r - l + 1
    if largest_k <= 1:
        return 
    
    # If the subarray is smaller than 100, run insertion sort.
    elif largest_k <= 100:
        
        # The algorith sorts from left to right, starting with 
        # the second number of the list.
        for j in range(l+1, largest_k):
            key = arr[j]

            # The key is now compared to the existing sorted way
            # (initialized as A[0]) And inserted into the proper position.
            # The loop iterates the sorted array from right to left.
            i = j - 1
            while i >= 0 and arr[i] > key:
                arr[i + 1] = arr[i]
                i = i - 1
            arr[i + 1] = key
        
    else:
        # If the user-defined k is larger than the size of the array, 
        # set k to the default.
        if largest_k < k:
            k = 2
        
        # Calculate the relevant mid points.
        mid_points = [((_*(r - l)) // k) + l for _ in range(1, k)]
        
        # Call mergeSort on the k subproblems.
        mergeSort(arr, l, mid_points[0], k)
        mergeSort(arr, mid_points[-1] + 1, r, k)       
        for mid_index in range(1, len(mid_points)):
            mergeSort(arr, mid_points[mid_index - 1] + 1, 
                      mid_points[mid_index], k)
        
        merge(arr, l, mid_points, r)       
        

### Running Time Experiments

Now, I analyze the practical worst-case running times of six algorithms: the regular (binary) merge sort, three-, five-, ten-way, and n-way merge sort (where n is the input size), and mixed sort. I will compare running times for fours orders of magnitude, starting at $1000$ and going up until $10^6$. For each point estimate, I present an average of 10 runs of the same algorithm with the same input to average out inconsistencies. 

In [3]:
# Prepare testing function and inputs
import time

def testIt(fun, extraArgs, iterations = 10):
    history = [] # Save running times
    for i in range(iterations):
        start_time = time.time()
        fun(*extraArgs)
        history.append(time.time() - start_time)
    
    # If the solution is correct
    if extraArgs[0] == sorted(extraArgs[0]):
        return sum(history)/iterations # Returning average running time
    else:
        print "Sorting algorithm is incorrect."


order3 = range(10**3, 0, -1)
order4 = range(10**4, 0, -1)
order5 = range(10**5, 0, -1)
order6 = range(10**6, 0, -1)
order7 = range(10**7, 0, -1)

In [4]:
# Run tests

print "Tests for binary merge sort."

print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order3, 0, len(order3) - 1, 2)), len(order3))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order4, 0, len(order4) - 1, 2)), len(order4))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order5, 0, len(order5) - 1, 2)), len(order5))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order6, 0, len(order6) - 1, 2)), len(order6))

print "Test for three-way merge sort."

print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order3, 0, len(order3) - 1, 3)), len(order3))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order4, 0, len(order4) - 1, 3)), len(order4))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order5, 0, len(order5) - 1, 3)), len(order5))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order6, 0, len(order6) - 1, 3)), len(order6))

print "Test for five-way merge sort."

print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order3, 0, len(order3) - 1, 5)), len(order3))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order4, 0, len(order4) - 1, 5)), len(order4))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order5, 0, len(order5) - 1, 5)), len(order5))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order6, 0, len(order6) - 1, 5)), len(order6))

print "Test for ten-way merge sort."

print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order3, 0, len(order3) - 1, 10)), len(order3))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order4, 0, len(order4) - 1, 10)), len(order4))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order5, 0, len(order5) - 1, 10)), len(order5))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order6, 0, len(order6) - 1, 10)), len(order6))

print "Test for n-way merge sort."

print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order3, 0, len(order3) - 1, len(order3))), len(order3))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mergeSort, (order4, 0, len(order4) - 1, len(order4))), len(order4))
# These take too long.
#print "Average running time: {:.3f} seconds for input size of {:d}".format(
#    testIt(mergeSort, (order5, 0, len(order5) - 1, len(order5))), len(order5))
#print "Average running time: {:.3f} seconds for input size of {:d}".format(
#    testIt(mergeSort, (order6, 0, len(order6) - 1, len(order6))), len(order6))

print "Test for augmented (mixed) merge sort."

print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mixedSort, (order3, 0, len(order3) - 1, 2)), len(order3))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mixedSort, (order4, 0, len(order4) - 1, 2)), len(order4))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mixedSort, (order5, 0, len(order5) - 1, 2)), len(order5))
print "Average running time: {:.3f} seconds for input size of {:d}".format(
    testIt(mixedSort, (order6, 0, len(order6) - 1, 2)), len(order6))
  


Tests for binary merge sort.
Average running time: 0.012 seconds for input size of 1000
Average running time: 0.183 seconds for input size of 10000
Average running time: 2.027 seconds for input size of 100000
Average running time: 21.349 seconds for input size of 1000000
Test for three-way merge sort.
Average running time: 0.014 seconds for input size of 1000
Average running time: 0.130 seconds for input size of 10000
Average running time: 1.612 seconds for input size of 100000
Average running time: 16.694 seconds for input size of 1000000
Test for five-way merge sort.
Average running time: 0.008 seconds for input size of 1000
Average running time: 0.118 seconds for input size of 10000
Average running time: 1.178 seconds for input size of 100000
Average running time: 15.026 seconds for input size of 1000000
Test for ten-way merge sort.
Average running time: 0.007 seconds for input size of 1000
Average running time: 0.087 seconds for input size of 10000
Average running time: 0.945 secon

| Merge Sort   | $10^3$   | $10^4$   | $10^5$   | $10^6$   |
|--------------|------|------|------|------|
|   2-way      | 0.012| 0.183| 2.027| 21.35|
|   3-way      | 0.014| 0.130| 1.612| 16.70|
|   5-way      | 0.008| 0.118| 1.178| 15.03|
|   10-way     | 0.007| 0.087| 0.945| 15.32|
|   n-way      | 0.150| 17.98|  -  |  -  |
|   Augmented  | 0.014| 0.175| 2.425| 21.92|

The table presents running times in seconds. As expected, the running time increases log-linearly with input size for small $k$. We have evidence that as $k$ increases, the economy in recursive calls to mergeSort seems to outsize the added complexity of the merge procedure until $k$ gets close to the input size $n$. However, we also see some fluctuations in this pattern which prevents us from arriving at a definitive conclusion. The question of the relationship between an optimal choice of $k$ and the input size $n$ is a direction for further research. It is certain that, as we stipulated, that $k = n$ renders the merge procedure $O(n^2)$ thus being a poor choice. We have mixed evidence on whether including insertion sort as a resort for sorting small sublists improves the running of regular merge sort, leaving the benefit inconclusive. Therefore, the optimal k-way sorting algorithm seems to be one in which $2 < k << n$.

### HC Application

\#algorithms: Effective implementation of a complex algorithm accompanied with appropriate, in-depth discussion. Code is effectively implemented and helpful comments are provided throughout. 